In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import sklearn.decomposition as decomposition
from sklearn.manifold import TSNE
from sklearn.metrics import f1_score, accuracy_score, make_scorer
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow.keras import Model, Input, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM, GRU, BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
#%load_ext tensorboard
#from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt
import seaborn as sb
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as mpatches

In [2]:
df_train = pd.read_csv("data/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("data/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

n_class = np.unique(Y).size

X.shape, Y.shape

((87554, 187, 1), (87554,))

### Parameter search on RNN

In [3]:
def build_gru(n_class=5, dropout=0.3, rnn_sizes = [128, 128], fc_sizes=[64], batch_norm=True):
    nclass = 5
    model = Sequential()
    model.add(Input(shape=(187, 1)))
    
    if batch_norm:
        model.add(BatchNormalization())
        
    for index, dim in enumerate(rnn_sizes):
        model.add(GRU(dim, dropout=dropout, return_sequences=(index != len(rnn_sizes) - 1)))
        
        if batch_norm:
            model.add(BatchNormalization())
    
    for index, dim in enumerate(fc_sizes):
        model.add(Dense(dim, activation="relu"))
        model.add(Dropout(dropout))
        
        if batch_norm:
            model.add(BatchNormalization())
            
    model.add(Dense(nclass, activation="softmax"))

    return model

In [4]:
class CustomRNN(BaseEstimator):
    def fit(self, train_X, train_y, **kwargs):
        
        self.build_model()
        
        # early = EarlyStopping(monitor="val_accuracy", mode="max", patience=5, verbose=1)
        # redonplat = ReduceLROnPlateau(monitor="val_accuracy", mode="max", patience=3, verbose=2)
        # callbacks_list = [checkpoint, early, redonplat]  # early
        self.model.fit(train_X, train_y, validation_split=0.1, epochs=self.epochs, batch_size=self.batch_size)
    
    def predict(self, eval_X):
        return np.argmax(self.model.predict(eval_X), axis=1)
    
    def set_params(self, epochs=100, 
                         batch_size=64, 
                         learning_rate=1e-3, 
                         dropout=0.3, 
                         rnn_sizes=[128, 128],
                         fc_sizes=[64],
                         batch_norm=True):
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.dropout = dropout
        self.rnn_sizes = rnn_sizes
        self.fc_sizes = fc_sizes
        self.batch_norm = batch_norm
                
        return self
        
    def build_model(self):
        self.model = build_gru(n_class=5, dropout=self.dropout, 
                          rnn_sizes=self.rnn_sizes, fc_sizes=self.fc_sizes, batch_norm=self.batch_norm)
        opt = optimizers.Adam(self.learning_rate)
            
        self.model.compile(optimizer=opt, 
                      loss="sparse_categorical_crossentropy", 
                      metrics=['accuracy'])        

In [ ]:
params = {
    'epochs': [2],
    'batch_size': [64],
    'learning_rate': [1e-3],
    'dropout': [0.2],
    'rnn_sizes': [[128, 128], [128, 128, 128]],
    'fc_sizes': [[64], [64, 64], [64, 32]],
    'batch_norm': [True, False]
}

dummy_params = {
    'epochs': [20],
    'batch_size': [64],
    'learning_rate': [1e-4],
    'dropout': [0.2],
    'rnn_sizes': [[128, 128]],
    'fc_sizes': [[64, 32]],
    'batch_norm': [True]
}

model = CustomRNN()
search = GridSearchCV(estimator=model, 
                      param_grid=dummy_params,
                      scoring=make_scorer(f1_score, average='macro'),
                      n_jobs=1, 
                      return_train_score=True, 
                      refit=False, 
                      verbose=10,
                      error_score='raise')
best = search.fit(X[:, :, :], Y[:])
best.__dict__

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] batch_norm=True, batch_size=64, dropout=0.2, epochs=20, fc_sizes=[64, 32], learning_rate=0.0001, rnn_sizes=[128, 128] 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Train on 63038 samples, validate on 7005 samples
Epoch 1/20
63038/63038 [==============================] - 79s 1ms/sample - loss: 1.6467 - accuracy: 0.3746 - val_loss: 1.3095 - val_accuracy: 0.8206
Epoch 2/20
63038/63038 [==============================] - 65s 1ms/sample - loss: 1.0468 - accuracy: 0.7165 - val_loss: 0.7695 - val_accuracy: 0.8213
Epoch 3/20
63038/63038 [==============================] - 64s 1ms/sample - loss: 0.8013 - accuracy: 0.8053 - val_loss: 0.6896 - val_accuracy: 0.8216
Epoch 4/20
63038/63038 [==============================] - 64s 1ms/sample - loss: 0.7361 - accuracy: 0.8176 - val_loss: 0.6492 - val_accuracy: 0.8217
Epoch 5/20
63038/63038 [==============================] - 64s 1ms/sample - loss: 0.6732 - accuracy: 0.8200 - val_loss: 0.5835 - val_accuracy: 0.8223
Epoch 6/20
63038/63038 [==============================] - 64s 1ms/sample - loss: 0.5891 - accuracy: 0.8272 - val_loss: 0.4873 - val_accuracy: 0.8444
Epoch 7/20
63038/63038 [==============================] -

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 23.6min remaining:    0.0s


Train on 63038 samples, validate on 7005 samples
Epoch 1/20
63038/63038 [==============================] - 82s 1ms/sample - loss: 1.6175 - accuracy: 0.3966 - val_loss: 1.1188 - val_accuracy: 0.8213
Epoch 2/20
14080/63038 [=====>........................] - ETA: 1:49 - loss: 1.2047 - accuracy: 0.6320